In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

Using TensorFlow backend.


In [2]:
# read data (for now, sell_prices & calendar are not used)

data_dir = 'data/'

train_sales = pd.read_csv(data_dir + 'sales_train_validation.csv')
#sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
#calendar = pd.read_csv(data_dir + 'calendar.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')

In [3]:
# create training data, for now it only contains the sales and no extra features
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1)
sales.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [4]:
# create X and y

# if you want to use all data, use start_day = 1

# because of memory issues, only use data from last year for now
start_day = 1913 - 365

timesteps = 14
prediction_steps = 28
len_window = timesteps + prediction_steps

nr_training_days = sales.shape[1] - start_day + 1
nr_sets = nr_training_days - len_window + 1

base, predictions = [], []

for i in range(nr_sets):
    if (start_day != 1):
        i = i + start_day - 1
    samples = sales.iloc[:, i:i+timesteps]
    preds = sales.iloc[:,i+timesteps:i+len_window]
    base.extend(samples.to_numpy())
    predictions.extend(preds.to_numpy())
    
base = np.array(base)
predictions = np.array(predictions)

Nu bestaat samples uit groepen van 14 opeenvolgende dagen, en predictions uit groepen van de 28 dagen erna. Dus het model leert de sales, voor alle producten, per 4 weken te voorspellen op basis van de sales van de 2 weken daarvoor. Kunnen ook proberen om per week te voorspellen, en dan de voorspellingen mee te nemen als input voor de volgende voorspelling. Of zelfs per dag.

Elke sample in de for-loop in de cell hierboven bevat de sales voor alle 30490 items van 14 opeenvolgende dagen. Omdat dus voor alle 30490 items tegelijk wordt voorspeld, kunnen we denk ik geen features toevoegen die iets over een item zeggen (zoals sell price, store, state, category etc), maar we kunnen wel features toevoegen die iets zeggen over een dag (zoals event, snap, month etc). Wat wel kan is voor ieder item een apart model trainen, 30490 models dus haha, maar dat lijkt me niet doable. Zouden misschien wel bijv per store een model kunnen trainen.

De sliding window werkt hier nu wel goed. Probleem is alleen dat er heel erg veel data komt, wat leidt tot memory errors. Als je alleen de sales van het laatste jaar in de data pakt (wat nu ook in de cell hierboven gebeurt), duurt het trainen van het model ongeveer al 2 uur.

start_day=1913-365, timesteps=14, prediction_steps=28 leidt tot score in kaggle van 0.97

In [5]:
# normalize & reshape data

input_scaler = MinMaxScaler()
output_scaler = StandardScaler()

input_scaler.fit(base)
X = input_scaler.transform(base)

X = X.reshape((X.shape[0], X.shape[1], 1))

output_scaler.fit(predictions)
y = output_scaler.transform(predictions)

In [9]:
del base, predictions

In [11]:
# create model

n_features = X.shape[2]

model = Sequential()
model.add(Bidirectional(LSTM(20, return_sequences=True, input_shape=(timesteps, n_features))))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(prediction_steps))
model.compile(optimizer='adam', loss='mse') # note that loss is not competition loss

In [12]:
# train model

model.fit(X, y, epochs=2, verbose=1) 

Epoch 1/2
9909250/9909250 [==============================] - 4123s 416us/step - loss: 0.4243
Epoch 2/2
9909250/9909250 [==============================] - 4121s 416us/step - loss: 0.4109


LSTM model is gemaakt en getrained. Nu nog predicten en submission maken. Omdat het model getrained is om 4 weken sales te voorspellen uit 2 weken daarvoor, wordt X_pred de laatste 2 weken van de data.

Also note: voorspellingen worden gedaan voor 'validation' (dus eerste 28 dagen na data in train). die voorspellingen worden direct gekopieerd naar 'evaluation' (28 dagen na validation), dus dat slaat nergens op. maar is niet erg want public leaderboard is alleen gebaseerd op validation.

In [13]:
# get input data for predictions

X_pred = sales.iloc[:,-timesteps:].to_numpy()

# reshape & normalize
X_pred = X_pred.reshape((len(sales), X_pred.shape[1]))
X_pred = input_scaler.transform(X_pred)
X_pred = X_pred.reshape((len(sales), X_pred.shape[1], 1))

# get predictions
norm_pred = model.predict(X_pred)
predictions = output_scaler.inverse_transform(norm_pred)
predictions = np.round(np.abs(predictions))

In [14]:
# create submission file

validation = pd.concat([pd.DataFrame(predictions[:,0:prediction_steps]), pd.DataFrame(predictions[:,-prediction_steps:])])
validation = validation.astype(int)

validation.reset_index(inplace=True, drop=True)

validation['id'] = submission_file.id
validation = validation.reindex(columns=['id'] + [c for c in validation.columns if c != 'id'], copy=False)

validation.columns = ['id'] + [f"F{i}" for i in range(1, 29)]

validation.to_csv('submission.csv', index=False)